<a href="https://colab.research.google.com/github/dlaywls/ML-DL/blob/main/Lab10_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import os
import glob
import cv2
from google.colab.patches import cv2_imshow

In [20]:
images='/content/drive/MyDrive/custom_data/origin_data'
image=cv2.imread(images)

In [34]:
train_data = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/custom_data/origin_data')

In [35]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out


폴더에 저장

In [ ]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)

    if(label == 0):
        data.save('/content/drive/MyDrive/custom_data/train_data/gray/%d_%d.jpeg'%(num, label))
    else:
        data.save('/content/drive/MyDrive/custom_data/train_data/red/%d_%d.jpeg'%(num, label))

testing

In [30]:
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [31]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [42]:
total_batch = len(data_loader)

epochs = 3
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

TypeError: ignored

In [45]:
torch.save(net.state_dict(), "/content/drive/MyDrive/model.pth")

In [47]:
new_net = CNN().to(device)

In [48]:
new_net.load_state_dict(torch.load('/content/drive/MyDrive/model.pth'))

<All keys matched successfully>

In [46]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/custom_data/test_data')

In [49]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))